If you see this, it's successfully commited

In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=78b327cfd368728c6757331a52e21f1d8c2e8ca4cbe64b127e70440bd60ec66e
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
import json

In [ ]:
spark = SparkSession.builder.getOrCreate()

In [ ]:
path = "/content/drive/MyDrive/Colab Notebooks/data/"

shows_log = spark.sparkContext.textFile(path + "shows-log")
clicks_log = spark.sparkContext.textFile(path + "clicks-log")
banners_dict = spark.sparkContext.textFile(path + "banners_dict")
banner_info = spark.sparkContext.textFile(path + "banner_info")
user_profile = spark.sparkContext.textFile(path + "user_profile")

ls = [shows_log, clicks_log, banners_dict, banner_info, user_profile]


In [ ]:
for i in banners_dict.collect():
    print(i)


Выходные данные были обрезаны до нескольких последних строк (5000).
4597222050	1958
1386710071	590
490057314	208
10949591264	4665
1166272671	496
4332696763	1846
6115304014	2605
3544030045	1510
7031206968	2996
205618273	87
14288163634	6088
9547548996	4068
4571085010	1947
6973097162	2971
14228174141	6062
10958535316	4669
12937703092	5512
1936453191	825
12943030007	5515
8344137205	3555
11492714983	4897
11004822874	4689
3171082627	1351
8947744224	3812
8969596253	3822
6416421324	2734
344859674	146
694857630	296
4271862965	1820
3403420238	1450
11160074104	4755
5767377731	2457
10706952734	4562
892240323	380
10135676430	4318
6748852444	2875
10803033196	4603
5135510436	2188
14727513094	6275
3827324731	1630
13953945149	5945
9771652968	4163
1098923790	468
3012116748	1283
4796440943	2043
5424571545	2311
7829305041	3336
13369524236	5696
2668985451	1137
14699521020	6263
3908282130	1665
5139227192	2189
6766153610	2883
1699781365	724
3373930326	1437
3576629221	1524
7068576792	3012
6740860954	2872
3790

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-25-5ef66acf7424>", line 2, in <cell line: 1>
    print(i)
  File "/usr/local/lib/python3.10/dist-packages/ipykernel/iostream.py", line 402, in write
    self.pub_thread.schedule(lambda : self._buffer.write(string))
  File "/usr/local/lib/python3.10/dist-packages/ipykernel/iostream.py", line 203, in schedule
    self._event_pipe.send(b'')
  File "/usr/local/lib/python3.10/dist-packages/zmq/sugar/socket.py", line 620, in send
    return super().send(data, flags=flags, copy=copy, track=track)
  File "zmq/backend/cython/socket.pyx", line 746, in zmq.backend.cython.socket.Socket.send
  File "zmq/backend/cython/socket.pyx", line 793, in zmq.backend.cython.socket.Socket.send
  File "zmq/backend/cython/socket.pyx", line 250, in zmq.backend.cython.socket._send_copy
  File "zmq/bac

TypeError: object of type 'NoneType' has no len()

In [ ]:
def convert_json_to_rdd(x):
    res = json.loads(x)
    return res

user_profile = user_profile.map(convert_json_to_rdd)

In [ ]:
def from_tab_to_rdd(data):
    return tuple(data.split("\t"))

def reverse_group_clicks(clicks):
    return (clicks[2], (clicks[1], clicks[0]))

def process(shows, clicks, banners_dict, banner_info, user_profile):
    # prepare data
    clicks = clicks.map(from_tab_to_rdd)
    banners_dict = banners_dict.map(from_tab_to_rdd)

    #step 1
    clicks = clicks.map(reverse_group_clicks)
    clicks_with_types = clicks.join(banners_dict)


    res = clicks_with_types
    return res # dt, event_type, uid, banner_id, banner_type_id, age, sex, pay_event_type, currency, real_cost.

In [ ]:
result = process(shows_log, clicks_log, banners_dict, banner_info, user_profile)
print(result)

PythonRDD[17] at RDD at PythonRDD.scala:53


In [ ]:
for i in result.collect():
    print(i)

Выходные данные были обрезаны до нескольких последних строк (5000).
('10684039804', (('113067450732', '1667291440'), '4552'))
('10684039804', (('224154052124', '1667296293'), '4552'))
('10684039804', (('688881396339', '1667297684'), '4552'))
('10684039804', (('423636018170', '1667299432'), '4552'))
('10684039804', (('432061929457', '1667319343'), '4552'))
('10684039804', (('710591867972', '1667323064'), '4552'))
('10684039804', (('147219183294', '1667325258'), '4552'))
('3820869204', (('370271377334', '1667260548'), '1628'))
('3820869204', (('11030185526', '1667296467'), '1628'))
('3820869204', (('692162331368', '1667302317'), '1628'))
('3820869204', (('668478853148', '1667331153'), '1628'))
('14935926334', (('707868190889', '1667260556'), '6364'))
('1266171977', (('516118305681', '1667260566'), '539'))
('1266171977', (('498191012361', '1667278586'), '539'))
('1266171977', (('559877830805', '1667313584'), '539'))
('1022949352', (('334790100601', '1667260584'), '435'))
('411279272', (('

KeyboardInterrupt: 

In [ ]:
spark.stop()